In [47]:
import numpy as np
import pandas as pd
import requests
from tqdm.notebook import tqdm
from libbgg.apiv1 import BGG
import itertools

In [48]:
bbg = 'data/bgg.csv'

In [49]:
df = pd.read_csv(bbg)

In [50]:
df.head()

,game,title,rating
0,25984,Trivial Pursuit: Genus IV – Canadian Edition,3.0
1,25984,Trivial Pursuit: Genus IV – Canadian Edition,5.0
2,25984,Trivial Pursuit: Genus IV – Canadian Edition,3.0
3,25984,Trivial Pursuit: Genus IV – Canadian Edition,7.0
4,25984,Trivial Pursuit: Genus IV – Canadian Edition,6.0


In [51]:
id_list = df.game.unique()
len(id_list)

23931

In [52]:
def chunks(iterable, size):
    """Generate adjacent chunks of data"""
    it = iter(iterable)
    return iter(lambda: tuple(itertools.islice(it, size)), ())


In [53]:
chunk = list(chunks(id_list, 1000))

In [54]:
conn = BGG()

In [55]:
games = []

In [56]:
for i in chunk:
    results = conn.get_game(game_ids=i , stats=True)
    
    for rec in results['boardgames']['boardgame']:
        record = {}
        #game id
        record['id'] = rec['objectid']
        #game name
        if not isinstance(rec['name'], list):
            record['name'] = rec['name']['TEXT']
        else:
            for i in rec['name']:
                try:
                    i['primary']
                    record['name'] =i['TEXT']
                except:
                    pass
        #game rank
        if not isinstance(rec['statistics']['ratings']['ranks']['rank'], list):
            rate=rec['statistics']['ratings']['ranks']['rank']
        else:
            rate=[value for index, value in
                   enumerate(rec['statistics']['ratings']['ranks']['rank'])
                   if value['name'] == 'boardgame'][0]
        record['rank']= rate['value']
        record['bayesaverage'] = rate['bayesaverage']
        games.append(record)


In [57]:
games_df = pd.DataFrame(games)

In [58]:
games_df.to_csv('data/bgg_true_stats.csv',index=False)

In [59]:
games_df.head()

,id,name,rank,bayesaverage
0,25984,Trivial Pursuit: Genus IV – Canadian Edition,Not Ranked,Not Ranked
1,103339,Reluctant Enemies: Operation Exporter,3416,5.82593
2,132758,Fool's Gold,3385,5.82964
3,16497,Rat Hot,3498,5.81678
4,195194,Horizons,3316,5.83852


In [60]:
games_df.describe()

,id,name,rank,bayesaverage
count,23923,23923,23923,23923
unique,23923,23370,17009,13626
top,94331,Blockade,Not Ranked,Not Ranked
freq,1,5,6915,6011
